In [ ]:
# -*- coding: utf-8 -*-
import requests
import fitz  # PyMuPDF
import streamlit as st
import io # Para manejo de flujos de bytes en memoria

# -----------------------------------------------------------------------------
# 1. INTEGRACIÓN CON EL CONTEXTO DE LA APP
# -----------------------------------------------------------------------------
# Recuperamos variables inyectadas por el motor de app.py
palette = locals().get("active_palette", ["#0576F3", "#36F48C", "#F47806"])
active_font = locals().get("active_font", "sans-serif")

# Esta variable viene directamente del Selectbox principal de la Pestaña 2 en app.py
target_state = locals().get("ESTADO_SELECCIONADO", "Ciudad de México")

# Diccionario de claves para la API de la SCT (Datos Viales)
STATES_MAP = {
    "Aguascalientes": ("01", "AGUASCALIENTES"),
    "Baja California": ("02", "BAJA_CALIFORNIA"),
    "Baja California Sur": ("03", "BAJA_CALIFORNIA_SUR"),
    "Campeche": ("04", "CAMPECHE"),
    "Coahuila": ("05", "COAHUILA"),
    "Colima": ("06", "COLIMA"),
    "Chiapas": ("07", "CHIAPAS"),
    "Chihuahua": ("08", "CHIHUAHUA"),
    "Ciudad de México": ("09", "CIUDAD_DE_MEXICO"),
    "Durango": ("10", "DURANGO"),
    "Guanajuato": ("11", "GUANAJUATO"),
    "Guerrero": ("12", "GUERRERO"),
    "Hidalgo": ("13", "HIDALGO"),
    "Jalisco": ("14", "JALISCO"),
    "México": ("15", "MEXICO"),
    "Michoacán": ("16", "MICHOACAN"),
    "Morelos": ("17", "MORELOS"),
    "Nayarit": ("18", "NAYARIT"),
    "Nuevo León": ("19", "NUEVO_LEON"),
    "Oaxaca": ("20", "OAXACA"),
    "Puebla": ("21", "PUEBLA"),
    "Querétaro": ("22", "QUERETARO"),
    "Quintana Roo": ("23", "QUINTANA_ROO"),
    "San Luis Potosí": ("24", "SAN_LUIS_POTOSI"),
    "Sinaloa": ("25", "SINALOA"),
    "Sonora": ("26", "SONORA"),
    "Tabasco": ("27", "TABASCO"),
    "Tamaulipas": ("28", "TAMAULIPAS"),
    "Tlaxcala": ("29", "TLAXCALA"),
    "Veracruz": ("30", "VERACRUZ"),
    "Yucatán": ("31", "YUCATAN"),
    "Zacatecas": ("32", "ZACATECAS")
}

# -----------------------------------------------------------------------------
# 2. LÓGICA DE PROCESAMIENTO EN MEMORIA
# -----------------------------------------------------------------------------

@st.cache_data(show_spinner=False)
def fetch_map_in_memory(state_name):
    """
    Descarga el PDF a RAM, lo convierte a imagen en RAM y retorna los bytes.
    """
    if state_name not in STATES_MAP:
        return None, "Estado no encontrado en el catálogo de mapas."

    code, name_api = STATES_MAP[state_name]
    url = f"https://micrs.sct.gob.mx/images/DireccionesGrales/DGST/Datos-Viales-2016/{code}_{name_api}.pdf"

    try:
        # 1. Descarga a Memoria
        response = requests.get(url, timeout=45)
        if response.status_code != 200:
            return None, f"Error HTTP {response.status_code} al acceder a: {url}"
        
        pdf_bytes = response.content 

        # 2. Procesamiento con PyMuPDF
        doc = fitz.open(stream=pdf_bytes, filetype="pdf")
        
        if doc.page_count < 1:
            return None, "El PDF descargado está vacío."

        page = doc.load_page(0)
        
        # Zoom x2 para buena calidad
        mat = fitz.Matrix(2.0, 2.0)
        pix = page.get_pixmap(matrix=mat, alpha=False)

        # 3. Convertir a PNG bytes
        img_bytes = pix.tobytes("png")
        doc.close()
        
        return img_bytes, None

    except Exception as e:
        return None, f"Error de procesamiento: {str(e)}"

# -----------------------------------------------------------------------------
# 3. INTERFAZ DE USUARIO
# -----------------------------------------------------------------------------

# Validación de seguridad: Verificar si el estado seleccionado existe en nuestro mapa
if target_state not in STATES_MAP:
    st.warning(f"⚠️ No hay mapa disponible para: {target_state}")
else:
    st.markdown(
        f"""
        <h3 style='font-family:{active_font}; color:{palette[0]}; margin-bottom: 10px;'>
        🛣️ Mapa de Conectividad: <span style='color: #333'>{target_state}</span>
        </h3>
        """, 
        unsafe_allow_html=True
    )

    col_btn, col_info = st.columns([1, 2])
    
    with col_btn:
        # Botón único de acción
        btn_cargar = st.button(f"📥 Obtener Mapa de {target_state}", type="primary", use_container_width=True)

    with col_info:
        if not btn_cargar:
            st.caption("Presiona el botón para descargar y visualizar el mapa carretero oficial de la SCT.")

    # Lógica de visualización
    if btn_cargar:
        st.divider()
        with st.spinner(f"Procesando mapa de {target_state} (esto puede tardar unos segundos)..."):
            img_data, err = fetch_map_in_memory(target_state)
        
        if err:
            st.error(f"❌ {err}")
        else:
            # Layout de resultados
            st.image(img_data, caption=f"Red carretera oficial - {target_state}", use_container_width=True)
            
            # Botón de descarga
            col_d1, col_d2 = st.columns([1, 3])
            with col_d1:
                st.download_button(
                    label="⬇️ Descargar PNG",
                    data=img_data,
                    file_name=f"Mapa_Carretero_{target_state.replace(' ', '_')}.png",
                    mime="image/png",
                    use_container_width=True
                )

ModuleNotFoundError: No module named 'fitz'